In [1]:
from __future__ import division
from __future__ import print_function
from linear_gae.evaluation import get_roc_score, clustering_latent_space
from linear_gae.kcore import compute_kcore, expand_embedding
from linear_gae.model import *
from linear_gae.optimizer import OptimizerAE, OptimizerVAE
from linear_gae.preprocessing import *
import numpy as np
import os
import scipy.sparse as sp
import tensorflow as tf
import time
import networkx as nx
import pickle as pkl
import sys
import pandas as pd

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

flags = tf.app.flags
FLAGS = flags.FLAGS

tf.app.flags.DEFINE_string('f', '', 'kernel')   # 添加的，不报错

In [3]:
# Select graph dataset
flags.DEFINE_string('dataset', 'citeseer', 'Name of the graph dataset')

# Select machine learning task to perform on graph
flags.DEFINE_string('task', 'link_prediction', 'Name of the learning task')

# Model
flags.DEFINE_string('model', 'gcn_ae', 'Name of the model')
''' Available Models:

- gcn_ae: Graph Autoencoder from Kipf and Welling (2016), with 2-layer
          GCN encoder and inner product decoder

- gcn_vae: Graph Variational Autoencoder from Kipf and Welling (2016), with
           Gaussian priors, 2-layer GCN encoders for mu and sigma, and inner
           product decoder

- linear_ae: Linear Graph Autoencoder, as introduced in section 3 of NeurIPS
             workshop paper, with linear encoder, and inner product decoder

- linear_vae: Linear Graph Variational Autoencoder, as introduced in section 3
              of NeurIPS workshop paper, with Gaussian priors, linear encoders
              for mu and sigma, and inner product decoder
 
- deep_gcn_ae: Deeper version of Graph Autoencoder, as introduced in section 4
               of NeurIPS workshop paper, with 3-layer GCN encoder, and inner
               product decoder
 
- deep_gcn_vae: Deeper version of Graph Variational Autoencoder, as introduced
                in section 4 of NeurIPS workshop paper, with Gaussian priors,
                3-layer GCN encoders for mu and sigma, and inner product
                decoder
'''

' Available Models:\n\n- gcn_ae: Graph Autoencoder from Kipf and Welling (2016), with 2-layer\n          GCN encoder and inner product decoder\n\n- gcn_vae: Graph Variational Autoencoder from Kipf and Welling (2016), with\n           Gaussian priors, 2-layer GCN encoders for mu and sigma, and inner\n           product decoder\n\n- linear_ae: Linear Graph Autoencoder, as introduced in section 3 of NeurIPS\n             workshop paper, with linear encoder, and inner product decoder\n\n- linear_vae: Linear Graph Variational Autoencoder, as introduced in section 3\n              of NeurIPS workshop paper, with Gaussian priors, linear encoders\n              for mu and sigma, and inner product decoder\n \n- deep_gcn_ae: Deeper version of Graph Autoencoder, as introduced in section 4\n               of NeurIPS workshop paper, with 3-layer GCN encoder, and inner\n               product decoder\n \n- deep_gcn_vae: Deeper version of Graph Variational Autoencoder, as introduced\n                

In [4]:
# Model parameters
flags.DEFINE_float('dropout', 0., 'Dropout rate (1 - keep probability).')
flags.DEFINE_integer('epochs', 200, 'Number of epochs in training.')
flags.DEFINE_boolean('features', False, 'Include node features or not in encoder')
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate (with Adam)')
flags.DEFINE_integer('hidden', 32, 'Number of units in GCN hidden layer(s).')
flags.DEFINE_integer('dimension', 16, 'Dimension of encoder output, i.e. \
                                       embedding dimension')

# Experimental setup parameters
flags.DEFINE_integer('nb_run', 10, 'Number of model run + test')
flags.DEFINE_float('prop_val', 5., 'Proportion of edges in validation set \
                                   (for Link Prediction task)')
flags.DEFINE_float('prop_test', 10., 'Proportion of edges in test set \
                                      (for Link Prediction task)')
flags.DEFINE_boolean('validation', False, 'Whether to report validation \
                                           results at each epoch (for \
                                           Link Prediction task)')
flags.DEFINE_boolean('verbose', True, 'Whether to print comments details.')

# Parameters related to the "degeneracy framework" from IJCAI 2019 paper,
# aiming at scaling-up Graph AE/VAE by training the model only on the k-core
# (smaller) version of the graph, then expanding embedding to remaining nodes
# via simpler and faster heuristics
flags.DEFINE_boolean('kcore', False, 'Whether to run k-core decomposition \
                                      and use the framework. False = model \
                                      will be trained on the entire graph')
flags.DEFINE_integer('k', 2, 'Which k-core to use. Higher k => smaller graphs\
                              and faster (but maybe less accurate) training')
flags.DEFINE_integer('nb_iterations', 10, 'Number of fix point iterations in \
                                           algorithm 2 of IJCAI paper. See \
                                           kcore.py file for details')

# Lists to collect average results
if FLAGS.task == 'link_prediction':
    mean_roc = []
    mean_ap = []

if FLAGS.kcore:
    mean_time_kcore = []
    mean_time_train = []
    mean_time_expand = []
    mean_core_size = []
mean_time = []

In [5]:
def load_data(filename_adj, nodes_numbers):
    raw_edges = pd.read_csv(filename_adj, header=None)
    
    drop_self_loop = raw_edges[raw_edges[0] != raw_edges[1]]
    
    graph_np = np.zeros((nodes_numbers, nodes_numbers))
    
    for i in range(drop_self_loop.shape[0]):
        graph_np[drop_self_loop.iloc[i,0], drop_self_loop.iloc[i,1]]=1
        graph_np[drop_self_loop.iloc[i,1], drop_self_loop.iloc[i,0]]=1
        
    adj = nx.adjacency_matrix(nx.from_numpy_matrix(graph_np))
    
    features = sp.lil_matrix(np.eye(nodes_numbers))
    
    return adj, features

In [6]:
# Load graph dataset
if FLAGS.verbose:
    print("Loading data...")

adj_name = 'datasets/citeseer-edges.txt'
nodes_number = 3327
adj_init, features_init = load_data(adj_name, nodes_number)

Loading data...


In [7]:
# The entire training+test process is repeated FLAGS.nb_run times
for i in range(FLAGS.nb_run):

    if FLAGS.task == 'link_prediction' :
        if FLAGS.verbose:
            print("Masking test edges...")
        # Edge Masking for Link Prediction: compute Train/Validation/Test set
        adj, val_edges, val_edges_false, test_edges, test_edges_false = \
        mask_test_edges(adj_init, FLAGS.prop_test, FLAGS.prop_val)

    # Start computation of running times
    t_start = time.time()

    # Degeneracy Framework / K-Core Decomposition
    if FLAGS.kcore:
        if FLAGS.verbose:
            print("Starting k-core decomposition of the graph")
        # Save adjacency matrix of un-decomposed graph
        # (needed to embed nodes that are not in k-core, after GAE training)
        adj_orig = adj
        # Get the (smaller) adjacency matrix of the k-core subgraph,
        # and the corresponding nodes
        adj, nodes_kcore = compute_kcore(adj, FLAGS.k)
        # Get the (smaller) feature matrix of the nb_core graph
        if FLAGS.features:
            features = features_init[nodes_kcore,:]
        # Flag to compute k-core decomposition's running time
        t_core = time.time()
    elif FLAGS.features:
        features = features_init

    # Preprocessing and initialization
    if FLAGS.verbose:
        print("Preprocessing and Initializing...")
    # Compute number of nodes
    num_nodes = adj.shape[0]
    # If features are not used, replace feature matrix by identity matrix
    if not FLAGS.features:
        features = sp.identity(adj.shape[0])
    # Preprocessing on node features
    features = sparse_to_tuple(features)
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]

    # Define placeholders
    placeholders = {
        'features': tf.sparse_placeholder(tf.float32),
        'adj': tf.sparse_placeholder(tf.float32),
        'adj_orig': tf.sparse_placeholder(tf.float32),
        'dropout': tf.placeholder_with_default(0., shape = ())
    }

    # Create model
    model = None
    if FLAGS.model == 'gcn_ae':
        # Standard Graph Autoencoder
        model = GCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'gcn_vae':
        # Standard Graph Variational Autoencoder
        model = GCNModelVAE(placeholders, num_features, num_nodes,
                            features_nonzero)
    elif FLAGS.model == 'linear_ae':
        # Linear Graph Autoencoder
        model = LinearModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'linear_vae':
        # Linear Graph Variational Autoencoder
        model = LinearModelVAE(placeholders, num_features, num_nodes,
                               features_nonzero)
    elif FLAGS.model == 'deep_gcn_ae':
        # Deep (3-layer GCN) Graph Autoencoder
        model = DeepGCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'deep_gcn_vae':
        # Deep (3-layer GCN) Graph Variational Autoencoder
        model = DeepGCNModelVAE(placeholders, num_features, num_nodes,
                                features_nonzero)
    else:
        raise ValueError('Undefined model!')

    # Optimizer
    pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
    norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0]
                                                - adj.sum()) * 2)
    with tf.name_scope('optimizer'):
        # Optimizer for Non-Variational Autoencoders
        if FLAGS.model in ('gcn_ae', 'linear_ae', 'deep_gcn_ae'):
            opt = OptimizerAE(preds = model.reconstructions,
                              labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                            validate_indices = False), [-1]),
                              pos_weight = pos_weight,
                              norm = norm)
        # Optimizer for Variational Autoencoders
        elif FLAGS.model in ('gcn_vae', 'linear_vae', 'deep_gcn_vae'):
            opt = OptimizerVAE(preds = model.reconstructions,
                               labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                             validate_indices = False), [-1]),
                               model = model,
                               num_nodes = num_nodes,
                               pos_weight = pos_weight,
                               norm = norm)

    # Normalization and preprocessing on adjacency matrix
    adj_norm = preprocess_graph(adj)
    adj_label = sparse_to_tuple(adj + sp.eye(adj.shape[0]))

    # Initialize TF session
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # Model training
    if FLAGS.verbose:
        print("Training...")

    for epoch in range(FLAGS.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        # Construct feed dictionary
        feed_dict = construct_feed_dict(adj_norm, adj_label, features,
                                        placeholders)
        feed_dict.update({placeholders['dropout']: FLAGS.dropout})
        # Weights update
        outs = sess.run([opt.opt_op, opt.cost, opt.accuracy],
                        feed_dict = feed_dict)
        # Compute average loss
        avg_cost = outs[1]
        if FLAGS.verbose:
            # Display epoch information
            print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
                  "time=", "{:.5f}".format(time.time() - t))
            # Validation, for Link Prediction
            if not FLAGS.kcore and FLAGS.validation and FLAGS.task == 'link_prediction':
                feed_dict.update({placeholders['dropout']: 0})
                emb = sess.run(model.z_mean, feed_dict = feed_dict)
                feed_dict.update({placeholders['dropout']: FLAGS.dropout})
                val_roc, val_ap = get_roc_score(val_edges, val_edges_false, emb)
                print("val_roc=", "{:.5f}".format(val_roc), "val_ap=", "{:.5f}".format(val_ap))

    # Flag to compute Graph AE/VAE training time
    t_model = time.time()

    # Compute embedding

    # Get embedding from model
    emb = sess.run(model.z_mean, feed_dict = feed_dict)

    # If k-core is used, only part of the nodes from the original
    # graph are embedded. The remaining ones are projected in the
    # latent space via the expand_embedding heuristic
    if FLAGS.kcore:
        if FLAGS.verbose:
            print("Propagation to remaining nodes...")
        # Project remaining nodes in latent space
        emb = expand_embedding(adj_orig, emb, nodes_kcore, FLAGS.nb_iterations)
        # Compute mean running times for K-Core, GAE Train and Propagation steps
        mean_time_expand.append(time.time() - t_model)
        mean_time_train.append(t_model - t_core)
        mean_time_kcore.append(t_core - t_start)
        # Compute mean size of K-Core graph
        # Note: size is fixed if task is node clustering, but will vary if
        # task is link prediction due to edge masking
        mean_core_size.append(len(nodes_kcore))

    # Compute mean total running time
    mean_time.append(time.time() - t_start)

    # Test model
    if FLAGS.verbose:
        print("Testing model...")
    # Link Prediction: classification edges/non-edges
    if FLAGS.task == 'link_prediction':
        # Get ROC and AP scores
        ap_score, roc_score = get_roc_score(test_edges, test_edges_false, emb)
        # Report scores
        mean_ap.append(ap_score)
        mean_roc.append(roc_score)

Masking test edges...
Preprocessing and Initializing...




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
targets is deprecated, use labels instead

Training...
Epoch: 0001 train_loss= 0.84164 time= 0.41252
Epoch: 0002 train_loss= 0.84051 time= 0.30361
Epoch: 0003 train_loss= 0.83776 time= 0.28184
Epoch: 0004 train_loss= 0.83254 time= 0.30246
Epoch: 0005 train_loss= 0.82391 time= 0.29300
Epoch: 0006 train_loss= 0.81105 time= 0.31743
Epoch: 0007 train_loss= 0.79359 time= 0.31597
Epoch: 0008 train_loss= 0.77234 time= 0.32023
Epoch: 0009 train_loss= 0.75013 time= 0.29885
Epoch: 0010 train_loss= 0.73273 time= 0.32687
Epoch: 0011 train_loss= 0.72711 time= 0.30498
Epoch: 0012 train_loss= 0.73065 time= 0.30410
Epoch: 0013 train_loss= 0.73008 time= 0.29562
Epoch: 0014 train_loss= 0.72152 time= 0.30520
Epoch: 0015 train_loss= 0.70872 time= 0.30882
Epoch: 0016 train_loss= 0.69594 time= 0.31665
Epoch

Epoch: 0131 train_loss= 0.43170 time= 0.29955
Epoch: 0132 train_loss= 0.43142 time= 0.32137
Epoch: 0133 train_loss= 0.43115 time= 0.30522
Epoch: 0134 train_loss= 0.43087 time= 0.30319
Epoch: 0135 train_loss= 0.43060 time= 0.31147
Epoch: 0136 train_loss= 0.43034 time= 0.29732
Epoch: 0137 train_loss= 0.43008 time= 0.32929
Epoch: 0138 train_loss= 0.42982 time= 0.29515
Epoch: 0139 train_loss= 0.42956 time= 0.30342
Epoch: 0140 train_loss= 0.42931 time= 0.30918
Epoch: 0141 train_loss= 0.42907 time= 0.29889
Epoch: 0142 train_loss= 0.42882 time= 0.31925
Epoch: 0143 train_loss= 0.42858 time= 0.31414
Epoch: 0144 train_loss= 0.42834 time= 0.31001
Epoch: 0145 train_loss= 0.42810 time= 0.30149
Epoch: 0146 train_loss= 0.42786 time= 0.29698
Epoch: 0147 train_loss= 0.42762 time= 0.30541
Epoch: 0148 train_loss= 0.42738 time= 0.30245
Epoch: 0149 train_loss= 0.42714 time= 0.31264
Epoch: 0150 train_loss= 0.42690 time= 0.33064
Epoch: 0151 train_loss= 0.42666 time= 0.31077
Epoch: 0152 train_loss= 0.42642 ti

Epoch: 0108 train_loss= 0.43143 time= 0.30664
Epoch: 0109 train_loss= 0.43115 time= 0.29714
Epoch: 0110 train_loss= 0.43087 time= 0.30167
Epoch: 0111 train_loss= 0.43059 time= 0.31553
Epoch: 0112 train_loss= 0.43032 time= 0.29938
Epoch: 0113 train_loss= 0.43004 time= 0.30002
Epoch: 0114 train_loss= 0.42977 time= 0.31414
Epoch: 0115 train_loss= 0.42951 time= 0.30020
Epoch: 0116 train_loss= 0.42924 time= 0.29163
Epoch: 0117 train_loss= 0.42898 time= 0.31836
Epoch: 0118 train_loss= 0.42871 time= 0.29427
Epoch: 0119 train_loss= 0.42845 time= 0.29726
Epoch: 0120 train_loss= 0.42819 time= 0.30446
Epoch: 0121 train_loss= 0.42793 time= 0.30019
Epoch: 0122 train_loss= 0.42767 time= 0.30745
Epoch: 0123 train_loss= 0.42741 time= 0.31356
Epoch: 0124 train_loss= 0.42714 time= 0.30373
Epoch: 0125 train_loss= 0.42688 time= 0.29699
Epoch: 0126 train_loss= 0.42662 time= 0.31226
Epoch: 0127 train_loss= 0.42635 time= 0.29421
Epoch: 0128 train_loss= 0.42608 time= 0.30319
Epoch: 0129 train_loss= 0.42581 ti

Epoch: 0085 train_loss= 0.44528 time= 0.27390
Epoch: 0086 train_loss= 0.44463 time= 0.27567
Epoch: 0087 train_loss= 0.44399 time= 0.27738
Epoch: 0088 train_loss= 0.44336 time= 0.28096
Epoch: 0089 train_loss= 0.44275 time= 0.27305
Epoch: 0090 train_loss= 0.44215 time= 0.29113
Epoch: 0091 train_loss= 0.44159 time= 0.26868
Epoch: 0092 train_loss= 0.44104 time= 0.30523
Epoch: 0093 train_loss= 0.44053 time= 0.29944
Epoch: 0094 train_loss= 0.44003 time= 0.30230
Epoch: 0095 train_loss= 0.43955 time= 0.27794
Epoch: 0096 train_loss= 0.43909 time= 0.28488
Epoch: 0097 train_loss= 0.43865 time= 0.27534
Epoch: 0098 train_loss= 0.43821 time= 0.29656
Epoch: 0099 train_loss= 0.43778 time= 0.29326
Epoch: 0100 train_loss= 0.43736 time= 0.27825
Epoch: 0101 train_loss= 0.43694 time= 0.28991
Epoch: 0102 train_loss= 0.43652 time= 0.27562
Epoch: 0103 train_loss= 0.43611 time= 0.27589
Epoch: 0104 train_loss= 0.43570 time= 0.29829
Epoch: 0105 train_loss= 0.43530 time= 0.29613
Epoch: 0106 train_loss= 0.43490 ti

Epoch: 0062 train_loss= 0.45319 time= 0.31918
Epoch: 0063 train_loss= 0.45234 time= 0.29464
Epoch: 0064 train_loss= 0.45151 time= 0.31003
Epoch: 0065 train_loss= 0.45070 time= 0.30959
Epoch: 0066 train_loss= 0.44990 time= 0.31719
Epoch: 0067 train_loss= 0.44911 time= 0.30437
Epoch: 0068 train_loss= 0.44832 time= 0.29846
Epoch: 0069 train_loss= 0.44755 time= 0.29812
Epoch: 0070 train_loss= 0.44680 time= 0.31219
Epoch: 0071 train_loss= 0.44609 time= 0.29834
Epoch: 0072 train_loss= 0.44542 time= 0.30493
Epoch: 0073 train_loss= 0.44479 time= 0.30100
Epoch: 0074 train_loss= 0.44418 time= 0.30298
Epoch: 0075 train_loss= 0.44361 time= 0.31017
Epoch: 0076 train_loss= 0.44306 time= 0.29325
Epoch: 0077 train_loss= 0.44253 time= 0.31268
Epoch: 0078 train_loss= 0.44202 time= 0.29871
Epoch: 0079 train_loss= 0.44153 time= 0.31394
Epoch: 0080 train_loss= 0.44106 time= 0.29442
Epoch: 0081 train_loss= 0.44061 time= 0.30285
Epoch: 0082 train_loss= 0.44018 time= 0.29721
Epoch: 0083 train_loss= 0.43977 ti

...

Epoch: 0195 train_loss= 0.41780 time= 0.30149
Epoch: 0196 train_loss= 0.41768 time= 0.30603
Epoch: 0197 train_loss= 0.41757 time= 0.33411
Epoch: 0198 train_loss= 0.41745 time= 0.31521
Epoch: 0199 train_loss= 0.41734 time= 0.32135
Epoch: 0200 train_loss= 0.41723 time= 0.30937
Testing model...
Masking test edges...
Preprocessing and Initializing...
Training...
Epoch: 0001 train_loss= 0.84166 time= 0.48472
Epoch: 0002 train_loss= 0.84065 time= 0.29672
Epoch: 0003 train_loss= 0.83825 time= 0.30576
Epoch: 0004 train_loss= 0.83371 time= 0.29373
Epoch: 0005 train_loss= 0.82614 time= 0.29186
Epoch: 0006 train_loss= 0.81466 time= 0.28943
Epoch: 0007 train_loss= 0.79876 time= 0.31238
Epoch: 0008 train_loss= 0.77880 time= 0.31163
Epoch: 0009 train_loss= 0.75684 time= 0.29604
Epoch: 0010 train_loss= 0.73748 time= 0.32414
Epoch: 0011 train_loss= 0.72757 time= 0.28871
Epoch: 0012 train_loss= 0.72903 time= 0.31799
Epoch: 0013 train_loss= 0.73034 time= 0.29780
Epoch: 0014 train_loss= 0.72367 time= 0.2

Epoch: 0172 train_loss= 0.41532 time= 0.31088
Epoch: 0173 train_loss= 0.41518 time= 0.30963
Epoch: 0174 train_loss= 0.41505 time= 0.31149
Epoch: 0175 train_loss= 0.41492 time= 0.30822
Epoch: 0176 train_loss= 0.41480 time= 0.31312
Epoch: 0177 train_loss= 0.41467 time= 0.33083
Epoch: 0178 train_loss= 0.41455 time= 0.31068
Epoch: 0179 train_loss= 0.41443 time= 0.32218
Epoch: 0180 train_loss= 0.41432 time= 0.31717
Epoch: 0181 train_loss= 0.41420 time= 0.29745
Epoch: 0182 train_loss= 0.41409 time= 0.29820
Epoch: 0183 train_loss= 0.41399 time= 0.29755
Epoch: 0184 train_loss= 0.41388 time= 0.30661
Epoch: 0185 train_loss= 0.41378 time= 0.31468
Epoch: 0186 train_loss= 0.41368 time= 0.29637
Epoch: 0187 train_loss= 0.41358 time= 0.29588
Epoch: 0188 train_loss= 0.41348 time= 0.31312
Epoch: 0189 train_loss= 0.41339 time= 0.30187
Epoch: 0190 train_loss= 0.41329 time= 0.33699
Epoch: 0191 train_loss= 0.41320 time= 0.30645
Epoch: 0192 train_loss= 0.41311 time= 0.30085
Epoch: 0193 train_loss= 0.41302 ti

Epoch: 0149 train_loss= 0.42133 time= 0.30639
Epoch: 0150 train_loss= 0.42112 time= 0.30499
Epoch: 0151 train_loss= 0.42091 time= 0.31030
Epoch: 0152 train_loss= 0.42071 time= 0.30947
Epoch: 0153 train_loss= 0.42050 time= 0.29442
Epoch: 0154 train_loss= 0.42030 time= 0.30917
Epoch: 0155 train_loss= 0.42010 time= 0.29421
Epoch: 0156 train_loss= 0.41990 time= 0.30237
Epoch: 0157 train_loss= 0.41970 time= 0.32089
Epoch: 0158 train_loss= 0.41950 time= 0.28757
Epoch: 0159 train_loss= 0.41930 time= 0.29409
Epoch: 0160 train_loss= 0.41910 time= 0.31815
Epoch: 0161 train_loss= 0.41890 time= 0.32413
Epoch: 0162 train_loss= 0.41870 time= 0.31483
Epoch: 0163 train_loss= 0.41850 time= 0.29904
Epoch: 0164 train_loss= 0.41830 time= 0.29494
Epoch: 0165 train_loss= 0.41811 time= 0.30580
Epoch: 0166 train_loss= 0.41791 time= 0.29617
Epoch: 0167 train_loss= 0.41772 time= 0.30318
Epoch: 0168 train_loss= 0.41753 time= 0.30818
Epoch: 0169 train_loss= 0.41734 time= 0.31639
Epoch: 0170 train_loss= 0.41716 ti

Epoch: 0126 train_loss= 0.42240 time= 0.29128
Epoch: 0127 train_loss= 0.42222 time= 0.29920
Epoch: 0128 train_loss= 0.42203 time= 0.30041
Epoch: 0129 train_loss= 0.42184 time= 0.30421
Epoch: 0130 train_loss= 0.42165 time= 0.31937
Epoch: 0131 train_loss= 0.42145 time= 0.31006
Epoch: 0132 train_loss= 0.42126 time= 0.31326
Epoch: 0133 train_loss= 0.42106 time= 0.30083
Epoch: 0134 train_loss= 0.42085 time= 0.30225
Epoch: 0135 train_loss= 0.42064 time= 0.30718
Epoch: 0136 train_loss= 0.42043 time= 0.31579
Epoch: 0137 train_loss= 0.42022 time= 0.30782
Epoch: 0138 train_loss= 0.42000 time= 0.29747
Epoch: 0139 train_loss= 0.41977 time= 0.30163
Epoch: 0140 train_loss= 0.41954 time= 0.31098
Epoch: 0141 train_loss= 0.41931 time= 0.30419
Epoch: 0142 train_loss= 0.41908 time= 0.29345
Epoch: 0143 train_loss= 0.41884 time= 0.30219
Epoch: 0144 train_loss= 0.41860 time= 0.29916
Epoch: 0145 train_loss= 0.41836 time= 0.30118
Epoch: 0146 train_loss= 0.41812 time= 0.31109
Epoch: 0147 train_loss= 0.41788 ti

Epoch: 0103 train_loss= 0.43082 time= 0.31140
Epoch: 0104 train_loss= 0.43060 time= 0.29582
Epoch: 0105 train_loss= 0.43038 time= 0.30467
Epoch: 0106 train_loss= 0.43015 time= 0.30353
Epoch: 0107 train_loss= 0.42993 time= 0.30901
Epoch: 0108 train_loss= 0.42970 time= 0.31617
Epoch: 0109 train_loss= 0.42947 time= 0.29641
Epoch: 0110 train_loss= 0.42924 time= 0.30174
Epoch: 0111 train_loss= 0.42901 time= 0.31407
Epoch: 0112 train_loss= 0.42878 time= 0.30319
Epoch: 0113 train_loss= 0.42855 time= 0.29920
Epoch: 0114 train_loss= 0.42831 time= 0.31479
Epoch: 0115 train_loss= 0.42808 time= 0.30512
Epoch: 0116 train_loss= 0.42785 time= 0.30241
Epoch: 0117 train_loss= 0.42762 time= 0.30291
Epoch: 0118 train_loss= 0.42738 time= 0.29424
Epoch: 0119 train_loss= 0.42715 time= 0.32994
Epoch: 0120 train_loss= 0.42692 time= 0.29521
Epoch: 0121 train_loss= 0.42669 time= 0.30732
Epoch: 0122 train_loss= 0.42646 time= 0.33413
Epoch: 0123 train_loss= 0.42622 time= 0.29880
Epoch: 0124 train_loss= 0.42599 ti

In [8]:
###### Report Final Results ######

# Report final results
print("\nTest results for", FLAGS.model,
      "model on", FLAGS.dataset, "on", FLAGS.task, "\n",
      "___________________________________________________\n")

if FLAGS.task == 'link_prediction':
    print("AP scores\n", mean_ap)
    print("Mean AP score: ", np.mean(mean_ap),
          "\nStd of AP scores: ", np.std(mean_ap), "\n \n")
    
    print("AUC scores\n", mean_roc)
    print("Mean AUC score: ", np.mean(mean_roc),
          "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")

if FLAGS.kcore:
    print("Details on degeneracy framework, with k =", FLAGS.k, ": \n \n")

    print("Running times for k-core decomposition\n", mean_time_kcore)
    print("Mean: ", np.mean(mean_time_kcore),
          "\nStd: ", np.std(mean_time_kcore), "\n \n")

    print("Running times for autoencoder training\n", mean_time_train)
    print("Mean: ", np.mean(mean_time_train),
          "\nStd: ", np.std(mean_time_train), "\n \n")

    print("Running times for propagation\n", mean_time_expand)
    print("Mean: ", np.mean(mean_time_expand),
          "\nStd: ", np.std(mean_time_expand), "\n \n")

    print("Sizes of k-core subgraph\n", mean_core_size)
    print("Mean: ", np.mean(mean_core_size),
          "\nStd: ", np.std(mean_core_size), "\n \n")


Test results for gcn_ae model on citeseer on link_prediction 
 ___________________________________________________

AP scores
 [0.8316411502813426, 0.8575565952550963, 0.8280541205613199, 0.8461098428691949, 0.8323484746913752, 0.846510282866165, 0.8333283590220779, 0.8298961679076655, 0.8121904210136766, 0.8452024107334349]
Mean AP score:  0.836283782520135 
Std of AP scores:  0.012116111162884972 
 

AUC scores
 [0.7719381717183915, 0.8073807511170148, 0.769803163869098, 0.7898949402246106, 0.7664774785653907, 0.7935756551141167, 0.7795000603791813, 0.7671150827194783, 0.7397826349474701, 0.7910783721772733]
Mean AUC score:  0.7776546310832025 
Std of AUC scores:  0.01798658159597936 
 

